# HW3
## Mahdi Sabbaghi (95109123)


## Problem 1:
### Part 1


First Reading the data

In [3]:
X= read.delim("model01.txt", header = FALSE, sep = ",");
X= as.matrix(X)

A function for "all but a node"!

In [4]:
except <-  function(m ,k){
    if (k==1){
        r<- c(2:m)
    } else if (k<m){
        r<- cbind(t(c(1:(k-1))), t(c((k+1):m)))
    } else {
        r<- c(1:(m-1))
    }
    r
}

A function that taking two vectors: indices and values and fill the indices with values!

In [5]:
filling <-  function(m, indices, values){
    X= rep(0, m);
    for (i in c(1:length(indices))){
        X[indices[i]]= values[i]; 
    }
    X
}

For Granger Causality, Because we know that the effects are at most by one delay of time unit, I used $Y_t^i\independent X_{t-1}^j|X_{t-1}^{-j}$
For this, I build myX and myY and then by thresholding the variance, I found the adjacenct Matric. After that I regressed the values on causes

In [6]:
Granger_causality <- function(data, thr){
    n= dim(data)[1]; 
    m= dim(data)[2];
    
    myX<- data[1:n-1, ];
    myY<- data[2:n  , ];
    
    adjacency<- matrix(0, nrow= m, ncol= m);
    for (j in c(1:m)){
        for (k in c(1:m)){
            mdl1<- lm(myY[ ,c(j)]~ myX[, except(m, k)]);
            res1<- residuals(mdl1);
            var1<- var(res1); 
            mdl2<- lm(myY[, c(j)]~ myX[, c(1:m)]);
            res2<- residuals(mdl2);
            var2<- var(res2); 
            if (var2< thr*var1){
                adjacency[j, k]=1;
            }
        }
    }
    adjacency
    for (j in c(1:m)){
        indices= which(adjacency[j, ]!= 0)
        mdl1<- lm(myY[ ,c(j)]~ myX[,indices]);
        coeff<- as.matrix(coef(mdl1))
        print(coeff)
        coeff<- coeff[2: (dim(coeff)[1])]
        print(coeff);
        adjacency[j, ]= filling(m, indices, coeff)
    }
    adjacency
}

Granger_causality(X, 0.999)

                        [,1]
(Intercept)       0.97251553
myX[, indices]V1  0.20172953
myX[, indices]V2 -0.06971603
myX[, indices]V3  0.50486260
[1]  0.20172953 -0.06971603  0.50486260
                       [,1]
(Intercept)       0.9672608
myX[, indices]V1 -0.3791335
myX[, indices]V2  0.2413680
[1] -0.3791335  0.2413680
                        [,1]
(Intercept)      -0.08553238
myX[, indices]V1 -0.09124049
myX[, indices]V2  0.29407473
myX[, indices]V3  0.30801015
[1] -0.09124049  0.29407473  0.30801015


0.20172953,-0.06971603,0.5048626
-0.37913354,0.24136804,0.0000000
-0.09124049,0.29407473,0.3080101


For PC Algorithm I just used the code I had from HW2 for the skeleton part. Also I make the initial edge for instantenous effect zero to have better results!  and then regress on the causes again

In [9]:
total_degree <- function(m){
  n= dim(m)[1]
  max=0 
  for (i in c(1:n)){
      s= sum(m[i, ])
      if (s>max){
          max= s
      }
  }
  max
}
vertex_degree <- function(m, i){
  s= sum(m[i, ])
  s
}
delete_edge <- function(m, i, j){
  m[i, j]<-0
  m[j, i]<-0
  m
}

In [10]:
correlation<- function(X, Y, alpha){
  n= length(X)
  rho= cor(X, Y)
  z <- 1/2*log((1+rho)/(1-rho))
  if (abs(z)*sqrt(n-3)> qnorm(1-alpha/2)){
    result<-0
  } else{
    result<-1
  }
  result
}

partial_correlation <- function(X, Y, Z, alpha){
  if (class(Z)== 'matrix'){
    m<- dim(Z)[2];
    n<- dim(Z)[1]; 
  } else{
    n <- length(Z)
    m<-1
  } 
  mm1 <- lm(X~Z)
  res1 <- mm1$residuals
  mm2 <- lm(Y~Z)
  res2 <- mm2$residuals
  rho <- cor(res1,res2)
  z <- 1/2*log((1+rho)/(1-rho))
  if (abs(z)*sqrt(n-m-3)> qnorm(1-alpha/2)){
    result<-0
  } else{
    result<-1
  }
  result
}

In [11]:
library(CombMSC)
PC_timeseries <- function(data, a){
    L= dim(data)[1]; 
    m= dim(data)[2];
    
    myX<- data[1:L-1, ];
    myY<- data[2:L  , ];

    X<- cbind(myX, myY);
    n<- dim(X)[2]
    
    zeros= matrix(0, nrow= m, ncol = m);
    ones=  matrix(1, nrow= m, ncol = m);
    
    m1= cbind(zeros, ones);
    m2= cbind(ones, zeros);
    
    adjacency<- rbind(m1, m2)
    l<-0
    repeat{
      for (i in c(1:n)){
        if (vertex_degree(adjacency, i)<l+1){
          next
        }
        for (j in which(adjacency[i, ]!= 0)){
          if(l==0){
            if (correlation(X[, i], X[, j], alpha= a)==1){
              adjacency<- delete_edge(adjacency, i, j)
            }
          } else{
            if (j<n){   
              sets<- cbind(t(which(adjacency[i, 1:j-1]!=0)), t(which(adjacency[i,(j+1):n]!=0)+j))
            } else{
              sets<- which(adjacency[i, 1:n-1]!=0)
            }
            d<- length(sets)
            subs= subsets(d, l, sets)
            if (class(subs)== 'matrix'){
              k= dim(subs)[1]
              for (p in 1:k){
                index= subs[p, ]
                if (partial_correlation(X[, i], X[, j], X[ ,index], alpha= a)==1){
                  adjacency<- delete_edge(adjacency, i, j)
                  break
                }
              }
            } else{
              if (partial_correlation(X[, i], X[, j], X[, subs], alpha= a)==1){
                  adjacency<- delete_edge(adjacency, i, j)
                  break
              }  
            }
          }
        }
      }
      l<- l+1
      if (total_degree(adjacency)<l+1){
        break
      }
    }

    adjacency<- t(adjacency[1:(n/2), (n/2+1):n])
    print(adjacency)
    
    for (j in c(1:m)){
        indices= which(adjacency[j, ]!= 0)
        mdl1<- lm(myY[ ,c(j)]~ myX[,indices]);
        coeff<- as.matrix(coef(mdl1))
        print(coeff)
        coeff<- coeff[2: (dim(coeff)[1])]
        print(coeff);
        adjacency[j, ]= filling(m, indices, coeff)
    }
    adjacency
}

PC_timeseries(X, 0.35)


Attaching package: 'CombMSC'


The following object is masked from 'package:stats':

    BIC




     [,1] [,2] [,3]
[1,]    1    1    1
[2,]    1    1    0
[3,]    1    1    1
                        [,1]
(Intercept)       0.97251553
myX[, indices]V1  0.20172953
myX[, indices]V2 -0.06971603
myX[, indices]V3  0.50486260
[1]  0.20172953 -0.06971603  0.50486260
                       [,1]
(Intercept)       0.9672608
myX[, indices]V1 -0.3791335
myX[, indices]V2  0.2413680
[1] -0.3791335  0.2413680
                        [,1]
(Intercept)      -0.08553238
myX[, indices]V1 -0.09124049
myX[, indices]V2  0.29407473
myX[, indices]V3  0.30801015
[1] -0.09124049  0.29407473  0.30801015


0.20172953,-0.06971603,0.5048626
-0.37913354,0.24136804,0.0000000
-0.09124049,0.29407473,0.3080101


#### For FIC we know that finding the skeleton is similar to PC

For LINGAM I used pcalg package

In [10]:
library(pcalg)
L= dim(X)[1]; 
m= dim(X)[2];
myX<- X[1:L-1, ];
myY<- X[2:L  , ];
data<- cbind(myX, myY);
n<- dim(data)[2]
dim(data)

res1 <- lingam(data, verbose = FALSE);

total= res1$Bpruned; total
Lingam_weights= total[(n/2+1):n, 1:(n/2)]; Lingam_weights

[1] 999   6

0.0000000,0.0000000,0.000000,0.000000,0,0.000000
0.6166413,0.0000000,1.516754,-3.518902,0,1.133455
0.0000000,0.0000000,0.000000,1.288400,0,0.000000
0.6623427,0.0000000,0.000000,0.000000,0,0.000000
-0.5462593,0.3216296,0.000000,0.000000,0,0.000000
-0.8699925,0.0000000,-1.019866,2.803620,0,0.000000


0.6623427,0.0000000,0.000000
-0.5462593,0.3216296,0.000000
-0.8699925,0.0000000,-1.019866


### Part 2:
It's trivial that LINGAM is terrible beacuse it doesn't know anything about the time series!
between Granger and PC, I prefer to use PC, beacuse the threshold for Granger was seriously High!

### Part 3:
By using PC:

In [13]:
Xp= read.delim("model02.txt", header = FALSE, sep = ",");
Xp= as.matrix(Xp)

PC_timeseries(Xp, 0.35)

     [,1] [,2] [,3]
[1,]    1    0    1
[2,]    1    1    0
[3,]    1    1    1
                      [,1]
(Intercept)      0.7941079
myX[, indices]V1 0.9322860
myX[, indices]V3 0.8925001
[1] 0.9322860 0.8925001
                        [,1]
(Intercept)       1.06950543
myX[, indices]V1 -0.18929479
myX[, indices]V2  0.07264126
[1] -0.18929479  0.07264126
                        [,1]
(Intercept)       0.04462369
myX[, indices]V1 -0.50525479
myX[, indices]V2  0.76810749
myX[, indices]V3 -0.39815088
[1] -0.5052548  0.7681075 -0.3981509


0.9322860,0.00000000,0.8925001
-0.1892948,0.07264126,0.0000000
-0.5052548,0.76810749,-0.3981509


by high probability, the weights are these:
(0.9, 0, 0.9;
-0.2, 0.1, 0;
-0.5, 0.8, -0.4)

## Problem 2:

I just implemented what the algorithm said!

In [1]:
ChainCom<- function(U, v){
    n= dim(U)[1]
    A= c(v);
    B= c(1:n); B= B[!B %in% A];
    G= U; 
    O= c();
    l=1;
    while (length(B)!= 0){
        print("Hi!")
        T= c();
        for (j in A){
            for (k in which(G[j, ]==1)){
                 if ((!k %in% T) && (k %in% B) ){
                     T= c(T, k);
                 }
            }
        }
        for (j in A){
            for (k in which(G[j, ]==1)){
                 if (k %in% T){
                     G[j, k]=2;
                     G[k, j]=3;
                 }
            }
        }
        repeat{
            print("Hello!")
            Gp= G;
            for (i in T){
                for(j in T){
                    if(Gp[i, j]==1){
                        for (k in c(1:n)){
                            if(Gp[k, i]==2 && Gp[k, j]==0){
                                Gp[i, j]=2;
                                Gp[j, i]=3;
                                break;
                            }
                        }
                    }
                }
            }
            if (all(Gp==G)){
                break
            }
            G= Gp;
        }
        A= T;
        B= setdiff(B, T);
        isolated= c();
        for (i in T){
            for(j in T){
                if(G[i, j]==1){
                    s= c(i, j);
                    isolated= union(isolated, s);
                }
            }
        }
        if(length(isolated)!= 0){
            set= G[isolated, isolated];
            O[[l]]= set;
            l<- l+1;
        }
        print(A)
        print(B)
        print(G)
        print(O)
    }
    list(G, O)
}

SizeMEC<- function(U){
    print("Disease!")
    p= dim(U)[1];
    n= sum(U)/2;
    if (n== (p-1)){
        r<- p;
    } else if (n== p){
        r<- 2*p;
    } else if (n== (p*(p-1)/2 -2)){
        r<- (p**2-p-4)*factorial(p-3);
    } else if (n== (p*(p-1)/2 -1)){
        r<- 2*factorial(p-1)- factorial(p-2);
    } else if (n== (p*(p-1)/2)){
        r<- factorial(p);
    } else{
        print("You!")
        Sum=0
        for (i in c(1:p)){
            O= ChainCom(U, i)[[2]];
            s=1
            for (j in c(1: length(O))){
                s= s*SizeMEC(O[[j]]);
            }
            Sum= Sum+s;
        }
        r<- Sum;
    }
    r
}

For the example 2 of original paper, I used my implementaton and it gives the right results!

In [2]:
U = matrix(c(0,1,1,0,0, 1,0,1,1,1, 1,1,0,0,1, 0,1,0,0,1, 0,1,1,1,0), nrow= 5, ncol= 5, byrow= TRUE);
SizeMEC(U)

[1] "Disease!"
[1] "You!"
[1] "Hi!"
[1] "Hello!"
[1] 2 3
[1] 4 5
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    2    2    0    0
[2,]    3    0    1    1    1
[3,]    3    1    0    0    1
[4,]    0    1    0    0    1
[5,]    0    1    1    1    0
[[1]]
     [,1] [,2]
[1,]    0    1
[2,]    1    0

[1] "Hi!"
[1] "Hello!"
[1] "Hello!"
[1] 4 5
integer(0)
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    2    2    0    0
[2,]    3    0    1    2    2
[3,]    3    1    0    0    2
[4,]    0    3    0    0    3
[5,]    0    3    3    2    0
[[1]]
     [,1] [,2]
[1,]    0    1
[2,]    1    0

[1] "Disease!"
[1] "Hi!"
[1] "Hello!"
[1] 1 3 4 5
integer(0)
     [,1] [,2] [,3] [,4] [,5]
[1,]    0    3    1    0    0
[2,]    2    0    2    2    2
[3,]    1    3    0    0    1
[4,]    0    3    0    0    1
[5,]    0    3    1    1    0
[[1]]
     [,1] [,2] [,3] [,4]
[1,]    0    1    0    0
[2,]    1    0    1    0
[3,]    0    1    0    1
[4,]    0    0    1    0

[1] "Disease!"
[1] "Hi!"
[1] "Hello!"
[1] 1

[1] 14

14 is the right answer!

### Problem 7 of HW#3-theory

I implemented the same as part of this HW1

In [36]:
n=1000
X= matrix(c(rnorm(1, 0, 1), rnorm(1, 0, 1), rnorm(1, 0, 1)), nrow=1, ncol=3);
X= rbind(X, c(8*X[1, 2]+ rnorm(1, 0, 1), rnorm(1, 0, 1), rnorm(1, 0, 1)))
for (t in c(3:n)){
    X= rbind(X, c(8*X[t-1, 2]+ rnorm(1, 0, 1), rnorm(1, 0, 1), 10*X[t-2, 2]+ rnorm(1, 0, 1)))
}